# Libraries 

In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import pickle
from sklearn.svm import SVC
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
from keras import Input
from keras.models import load_model

# read data set 

In [6]:
raw_mail_data = pd.read_csv('mail_data.csv')

In [7]:
raw_mail_data

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


# replace the null values with a null string

# Pre-Processing

In [10]:
mail_data = raw_mail_data.where((pd.notnull(raw_mail_data)),'')

In [11]:
mail_data.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [12]:
mail_data.shape

(5572, 2)

# Label Encoding

# label spam mail as 0;  ham mail as 1;

In [15]:
mail_data.loc[mail_data['Category'] == 'spam', 'Category',] = 0
mail_data.loc[mail_data['Category'] == 'ham', 'Category',] = 1

# separating the data as texts and label

In [17]:
X = mail_data['Message']

Y = mail_data['Category']

In [18]:
X

0       Go until jurong point, crazy.. Available only ...
1                           Ok lar... Joking wif u oni...
2       Free entry in 2 a wkly comp to win FA Cup fina...
3       U dun say so early hor... U c already then say...
4       Nah I don't think he goes to usf, he lives aro...
                              ...                        
5567    This is the 2nd time we have tried 2 contact u...
5568                 Will ü b going to esplanade fr home?
5569    Pity, * was in mood for that. So...any other s...
5570    The guy did some bitching but I acted like i'd...
5571                           Rofl. Its true to its name
Name: Message, Length: 5572, dtype: object

In [19]:
Y

0       1
1       1
2       0
3       1
4       1
       ..
5567    0
5568    1
5569    1
5570    1
5571    1
Name: Category, Length: 5572, dtype: object

# Splitting the data into training data & test data

In [21]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=3)

In [22]:
print(X.shape)
print(X_train.shape)
print(X_test.shape)

(5572,)
(4457,)
(1115,)


# Feature Extraction 

# transform the text data to feature vectors that can be used as input to the Logistic regression

In [25]:
feature_extraction = TfidfVectorizer(min_df = 1, stop_words='english', lowercase=True)
X_train_features = feature_extraction.fit_transform(X_train)
X_test_features = feature_extraction.transform(X_test)

convert Y_train and Y_test values as integers

In [27]:
Y_train = Y_train.astype('int')
Y_test = Y_test.astype('int')

In [28]:
X_train

3075                  Don know. I did't msg him recently.
1787    Do you know why god created gap between your f...
1614                         Thnx dude. u guys out 2nite?
4304                                      Yup i'm free...
3266    44 7732584351, Do you want a New Nokia 3510i c...
                              ...                        
789     5 Free Top Polyphonic Tones call 087018728737,...
968     What do u want when i come back?.a beautiful n...
1667    Guess who spent all last night phasing in and ...
3321    Eh sorry leh... I din c ur msg. Not sad alread...
1688    Free Top ringtone -sub to weekly ringtone-get ...
Name: Message, Length: 4457, dtype: object

In [29]:
X_train_features

<4457x7431 sparse matrix of type '<class 'numpy.float64'>'
	with 34775 stored elements in Compressed Sparse Row format>

# Training the Model

# Logistic Regression

In [32]:
model = LogisticRegression()

# training the Logistic Regression model with the training data

In [34]:
model.fit(X_train_features, Y_train)

LogisticRegression()

# Evaluating the trained model

# prediction on training data

In [37]:
prediction_on_training_data = model.predict(X_train_features)
accuracy_on_training_data = accuracy_score(Y_train, prediction_on_training_data)

In [38]:
print('Accuracy on training data : ', accuracy_on_training_data)

Accuracy on training data :  0.9676912721561588


# prediction on test data

In [40]:
prediction_on_test_data = model.predict(X_test_features)
accuracy_on_test_data = accuracy_score(Y_test, prediction_on_test_data)

In [41]:
print('Accuracy on test data : ', accuracy_on_test_data)

Accuracy on test data :  0.9668161434977578


# Save model

# Save vectorizer

In [44]:
with open("model_lr.pkl", "wb") as f:
    pickle.dump(model, f)
with open("vectorizer.pkl", "wb") as f:
    pickle.dump(feature_extraction, f)


# Train ANN Model


In [46]:
X_train_ann = X_train_features.toarray()
X_test_ann = X_test_features.toarray()

In [47]:
ann_model = Sequential()
ann_model.add(Input(shape=(X_train_ann.shape[1],)))  
ann_model.add(Dense(64, activation='relu'))
ann_model.add(Dense(32, activation='relu'))
ann_model.add(Dense(1, activation='sigmoid'))

In [48]:
ann_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
ann_model.fit(X_train_ann, Y_train, epochs=5, batch_size=32, verbose=1)

Epoch 1/5
140/140 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.8784 - loss: 0.4698
Epoch 2/5
140/140 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9883 - loss: 0.0632
Epoch 3/5
140/140 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9979 - loss: 0.0162
Epoch 4/5
140/140 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9992 - loss: 0.0058
Epoch 5/5
140/140 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9999 - loss: 0.0022


In [49]:
loss, accuracy = ann_model.evaluate(X_test_ann, Y_test, verbose=0)
print("ANN Accuracy on test data:", accuracy)

ANN Accuracy on test data: 0.9874439239501953


# Save ANN model

In [51]:
ann_model.save("model_ann.keras")  

# Predict 

In [ ]:
def check_with_lr(email):
    model = pickle.load(open("model_lr.pkl", "rb"))
    vectorizer = pickle.load(open("vectorizer.pkl", "rb"))
    features = vectorizer.transform([email])
    prediction = model.predict(features)[0]
    return "Not Spam ✅" if prediction == 1 else "Spam ❌"

def check_with_ann(email):
    model = load_model("model_ann.keras")
    vectorizer = pickle.load(open("vectorizer.pkl", "rb"))
    features = vectorizer.transform([email]).toarray()
    prediction = model.predict(features)[0][0]
    return "Not Spam ✅" if prediction > 0.5 else "Spam ❌"

# Get user input
if __name__ == "__main__":
    print("=== Email Spam Classifier ===")

    user_input_1 = input("\nEnter email text to check with Logistic Regression:\n> ")
    print("Logistic Regression Prediction:", check_with_lr(user_input_1))

    user_input_2 = input("\nEnter email text to check with ANN:\n> ")
    print("ANN Prediction:", check_with_ann(user_input_2))

=== Email Spam Classifier ===
